In [1]:
from pyspark.sql import *

#create spark session
spark = (SparkSession
        .builder
        .appName("kafka-streaming")
         .config("spark.jars.packages",
                 "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0")
         .getOrCreate())

In [2]:
spark


In [3]:
#read message from kafka stream
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "ride-bookings") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [5]:
#create a temp view of the kafka msg

df.createOrReplaceTempView("message")

In [6]:
#view information in view

spark.sql("SELECT * FROM message")

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [7]:
#write out the message to the console of the environment

res = spark.sql("SELECT * FROM message")
res.writeStream.format("console")\
    .outputMode("append")\
    .start()

In [ ]:
# Write the message back into Kafka in another topic
df_output = df \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "spark-output") \
  .option("checkpointLocation", "/tmp") \
  .start() \
  .awaitTermination()